In [ ]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
# Login to wandb. Replace "your_api_key_here" with your actual WANDB API key.
# Alternatively, you can set the WANDB_API_KEY environment variable.
wandb.login(key="your_api_key_here")

from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import optuna
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 17.4 MB/s eta 0:00:00


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Load and preprocess the dataset
# Load the "raw_review_CDs_and_Vinyl" configuration from the Amazon Reviews 2023 dataset
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_CDs_and_Vinyl", trust_remote_code=True)

# Since the dataset only has one split ("full"), first shuffle the data first and then select 100%
shuffled_dataset = dataset["full"].shuffle(seed=42)
subset_size = int(1 * len(shuffled_dataset))
subset_dataset = shuffled_dataset.select(range(subset_size))

# Filter out samples where rating equals 3
subset_dataset = subset_dataset.filter(lambda x: x["rating"] != 3)

# Initialize the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Define the tokenization and formatting function
def tokenize_and_format(examples):
    # Tokenize the 'text' field
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    # Create binary labels: 1 if rating > 3 (positive), 0 if rating < 3 (negative)
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Apply the tokenization function to the subset dataset using batched processing
tokenized_dataset = subset_dataset.map(tokenize_and_format, batched=True)

# Save a checkpoint of the tokenized dataset so you can load it in a new runtime later.
tokenized_dataset.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")

# Split the processed dataset into training and testing sets (80/20 split)
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

CDs_and_Vinyl.jsonl:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Map:   0%|          | 0/4543582 [00:00<?, ? examples/s]

Saving the dataset (0/17 shards):   0%|          | 0/4543582 [00:00<?, ? examples/s]

In [ ]:
# Step 2':To continue from here in a new Colab runtime, you can load the dataset as follows:
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_CDs_checkpoint")

# Split the loaded dataset into train and test sets (80/20 split)
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch")

Loading dataset from disk:   0%|          | 0/17 [00:00<?, ?it/s]

In [ ]:
# Step 3A: Hyperparameter search and Trainer setup

# Define the model initialization function for hyperparameter search
def model_init():
    return RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Define the compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Define the hyperparameter search space function using Optuna
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16])
    }

# Define initial TrainingArguments.
# Include WANDB integration with report_to and run_name settings.
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,                   # Initial learning rate; will be tuned
    per_device_train_batch_size=32,        # Initial batch size; will be tuned
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=8,
    warmup_steps=500,
    report_to=["wandb"],                 # Enable logging to Weights & Biases
    run_name="roberta_amazon_reviews",   # Experiment run name in wandb
    logging_steps=50,
    logging_first_step=True
)

# Calculate the total number of samples in the train and test datasets
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])

print("Total training samples:", total_train)
print("Total evaluation samples:", total_eval)

# Use partial data for training and evaluation
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Create a Trainer instance for hyperparameter search (using a subset of data for demonstration)
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Run hyperparameter search using Optuna backend (n_trials=10 for demonstration)
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=10,
    hp_space=hp_space,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Total training samples: 3634865
Total evaluation samples: 908717


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-03-30 07:08:51,493] A new study created in memory with name: no-name-1b7bb9cb-4300-40dc-8243-2062d6c95356
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max 

train/epoch,▁█
train/global_step,▁█
train/grad_norm,█▁
train/learning_rate,▁█
train/loss,█▁
train/epoch,0.02201
train/global_step,50
train/grad_norm,3.48189
train/learning_rate,1e-05
train/loss,0.666


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.160200,0.099550,0.963676,0.958927,0.961696,0.963676
2,0.126000,0.218929,0.940561,0.947857,0.963635,0.940561
3,0.070900,0.110802,0.972482,0.970494,0.971099,0.972482


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.154700,0.092442,0.965327,0.963438,0.962927,0.965327
2,0.125500,0.190850,0.945515,0.951499,0.964157,0.945515
3,0.066600,0.128071,0.975234,0.974784,0.974509,0.975234


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[I 2025-03-30 08:06:59,652] Trial 8 finished with value: 3.899761068758609 and parameters:

eval/accuracy,▆▁█
eval/f1,▅▁█
eval/loss,▁█▄
eval/precision,▁▂█
eval/recall,▆▁█
eval/runtime,▂█▁
eval/samples_per_second,▇▁█
eval/steps_per_second,▇▁█
train/epoch,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/global_step,▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
train/grad_norm,▂▂▁▁▁▂▁▃▂▃▇▁▁▁▃▃▁▃▃▁▁▁▁▁█▁▁▅


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.156700,0.079442,0.970281,0.970725,0.971287,0.970281
2,0.109500,0.111875,0.944964,0.950718,0.962372,0.944964
3,0.073200,0.072269,0.980187,0.979526,0.979474,0.980187
4,0.031100,0.111284,0.980187,0.979526,0.979474,0.980187


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: T

In [ ]:
# 3B. Print the best hyperparameters found by hyperparameter search
print("Best run hyperparameters:", best_run.hyperparameters)

# Update TrainingArguments with the best hyperparameters found
updated_training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_run.hyperparameters["learning_rate"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to=["wandb"],
    run_name="roberta_amazon_reviews",
    logging_steps=50
)

# Calculate the total number of samples in the train and test datasets
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])

# Use partial data for training and evaluation
train_subset_final = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.002 * total_train)))
eval_subset_final = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.002 * total_eval)))

# Create the final Trainer instance with updated TrainingArguments for final training
final_trainer = Trainer(
    model_init=model_init,
    args=updated_training_args,
    train_dataset=train_subset_final,
    eval_dataset=eval_subset_final,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Best run hyperparameters: {'learning_rate': 1.8239686998854285e-05, 'per_device_train_batch_size': 8}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 4: Start final training with the updated Trainer using the best hyperparameters
train_result = final_trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.175200,0.102172,0.972482,0.971312,0.971055,0.972482
2,0.074300,0.131787,0.971381,0.972111,0.973192,0.971381
3,0.080700,0.135196,0.976885,0.976217,0.975993,0.976885


In [ ]:
# Save the final model checkpoint after training
final_trainer.save_model("/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_CDs")